# Desafio: Enriquecendo os Dados com Pandas e Numpy

- Apenas enriqueci os dados adicionando uma coluna nova "Continente" unsando np.where.

- carreguei um csv desde o computador, enriqueci os dados e simulei o envio para o BigQuery. Também o exportei para csv

In [1]:
# Imports
import pandas as pd
import numpy as np
import pandas_gbq # Conexão com BigQuery pelo Pandas
from google.oauth2 import service_account # Conexão com Google Cloud
from google.cloud import bigquery

In [2]:
# Esse seria o código para conectar, autorizar o google platform. Não vamos de fato usar aqui, apenas simular.

'''key_path = "xxx.json"

credentials = service_account.Credentials.from_service_account_file(
    key_path, scopes=['https://www.xxxxxxxx.com/auth/cloud-platform']
)'''

'key_path = "xxx.json"\n\ncredentials = service_account.Credentials.from_service_account_file(\n    key_path, scopes=[\'https://www.xxxxxxxx.com/auth/cloud-platform\']\n)'

In [3]:
# Vamos ler os dados em csv
df_vinhos = pd.read_csv('Vinhos - Base_Vinhos.csv')
df_vinhos.head()

,Data compra,Data consumo,Valor,Vinho,Vinícula,Tipo,Uva,Graduação,Ano,Região,País,Loja,Ocasião,Avaliação,Motivo avaliação,Obs:
0,17/02/2023,21/02/2023,"32,9",Guapi Superior,Bodegas Tagua Tagua,Rosé,Merlot,"13,0%",2020.0,Vale do Rappel,Chile,O Comerciante,primeiro vinho,3,"Fácil de tomar, agradável",NaN
1,23/02/2023,26/02/2023,46,Sottano,Bodega Sottano,Rosé,Malbec,"13,5%",2022.0,Mendoza,Argentina,O Comerciante,segundo vinho,4,"Fácil de tomar, agradável",NaN
2,23/02/2023,02/06/2023,46,Piedra Rara,Piedra Rara,Tinto,Malbec,"13,4%",2021.0,Mendoza,Argentina,O Comerciante,Em casa,3,"Difícil de tomar, mais tempo no cálice",NaN
3,23/02/2023,08/05/2023,48,Le Mal Du Bec,Bodega Benoit,Tinto,Malbec,"13,5%",2020.0,Mendoza,Argentina,O Comerciante,Assado Fred,"3,5","Fácil de tomar, agradável",NaN
4,23/02/2023,20/05/2023,54,Carrascal,Cavas de Weinert,Branco,Chardonnay,"13,5%",2021.0,Mendoza,Argentina,O Comerciante,"Pizza Renata, Franco","4,5","Muito fácil de tomar, saboroso",NaN


In [4]:
# Visualiza Infos
df_vinhos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Data compra       13 non-null     object 
 1   Data consumo      13 non-null     object 
 2   Valor             13 non-null     object 
 3   Vinho             13 non-null     object 
 4   Vinícula          13 non-null     object 
 5   Tipo              13 non-null     object 
 6   Uva               13 non-null     object 
 7   Graduação         13 non-null     object 
 8   Ano               12 non-null     float64
 9   Região            13 non-null     object 
 10  País              13 non-null     object 
 11  Loja              13 non-null     object 
 12  Ocasião           13 non-null     object 
 13  Avaliação         13 non-null     object 
 14  Motivo avaliação  13 non-null     object 
 15  Obs:              1 non-null      object 
dtypes: float64(1), object(15)
memory usage: 1.8+ K

In [5]:
# Converte Valor e Avaliação em float
df_vinhos['Valor'] = df_vinhos['Valor'].str.replace(',', '.').astype('float')
df_vinhos['Avaliação'] = df_vinhos['Avaliação'].str.replace(',', '.').astype('float')

In [6]:
# Nesse rápido exercício, vamos apenas adicionar o continente de origem do vinho, com base nos unique_values.
df_vinhos['País'].unique()

array(['Chile', 'Argentina', 'Portugal', 'Brasil', 'Espanha'],
      dtype=object)

In [7]:
# Cria a coluna "Continente" usando np.where
df_vinhos['Continente'] = np.where(df_vinhos['País'].str.contains("Argentina|Chile|Brasil", case=False), 'América do Sul',
                                  np.where(df_vinhos['País'].str.contains("Portugal|Espanha", case=False), 'Europa', np.nan))

In [8]:
# Visualiza o shape
df_vinhos.shape

(13, 17)

### Rápida análise exploratória

In [9]:
# Groupby apenas como análise exploratória
df_vinhos.groupby('Continente').agg({'Vinho':'count',
                                    'Avaliação' : np.mean,
                                    'Valor' : np.mean})

,Vinho,Avaliação,Valor
Continente,,,
América do Sul,11,3.681818,55.081818
Europa,2,3.500000,59.500000


In [10]:
df_vinhos.groupby('Tipo').agg({'Vinho':'count',
                                    'Avaliação' : np.mean,
                                    'Valor' : np.mean})

,Vinho,Avaliação,Valor
Tipo,,,
Branco,3,4.333333,60.000000
Rosé,3,3.500000,38.300000
Tinto,7,3.428571,61.428571


In [11]:
df_vinhos.groupby('País').agg({'Vinho':'count',
                                    'Avaliação' : np.mean,
                                    'Valor' : np.mean})

,Vinho,Avaliação,Valor
País,,,
Argentina,6,3.916667,58.000
Brasil,1,3.500000,58.000
Chile,4,3.375000,49.975
Espanha,1,2.500000,37.000
Portugal,1,4.500000,82.000


### Loading dos dados
Simula envio para o BigQuery e exporta para csv

In [12]:
# Apenas simulação: Enviando a base atualizada para o Google Big Query, para alimentar o Looker Studio.
'''df_vinhos.to_gbq(credentials=credentials,
                                 destination_table='vinhos.df_vinhos',
                                 if_exists='replace')'''

# Exporta para csv
df_vinhos.to_csv('df_vinhos.csv', index=False)